In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18
from PIL import Image

# ✅ Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Normalize to standard ImageNet stats (common practice)
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

# ✅ Transform for both RGB and Grayscale images
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB")),  # Convert BW to RGB
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

# ✅ Load dataset from folders
train_dataset = datasets.ImageFolder("data/train", transform=transform)
test_dataset  = datasets.ImageFolder("data/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ✅ Get number of classes
num_classes = len(train_dataset.classes)

# ✅ Define a simple CNN
class MoodCNN(nn.Module):
    def __init__(self, num_classes):
        super(MoodCNN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.network(x)

# ✅ Initialize model
model = MoodCNN(num_classes=num_classes).to(device)

# ✅ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f} - Accuracy: {acc:.4f}")

# ✅ Save model
torch.save(model.state_dict(), "models/mood_cnn.pth")
print("✅ Training complete. Model saved.")


Epoch 1/10 - Loss: 4506.1047 - Accuracy: 0.4883
Epoch 2/10 - Loss: 3738.5291 - Accuracy: 0.5894
Epoch 3/10 - Loss: 3266.1809 - Accuracy: 0.6415
Epoch 4/10 - Loss: 2840.9724 - Accuracy: 0.6871
Epoch 5/10 - Loss: 2474.4162 - Accuracy: 0.7247
Epoch 6/10 - Loss: 2193.3073 - Accuracy: 0.7539
Epoch 7/10 - Loss: 1955.4946 - Accuracy: 0.7765
Epoch 8/10 - Loss: 1776.7031 - Accuracy: 0.7957
Epoch 9/10 - Loss: 1626.8220 - Accuracy: 0.8115
Epoch 10/10 - Loss: 1525.6492 - Accuracy: 0.8217
✅ Training complete. Model saved.
